# Your first scraper

- Repository: `Wikipedia Scrapper`
- Type of Challenge: `Learning`
- Duration: `3 days`
- Submission: `23/06/2023 4:30 PM`

- Team challenge : `Solo`

## The Mission

In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

## Learning objectives

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

* Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.
* Activate it.
* You will find more info about virtual environments in the course content and on the web.
* Note: BEFORE pushing changes into your repo explore what the .gitignore file is and add your environment to it. 

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [1]:
# import the requests library (1 line)
import requests
# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = 'https://country-leaders.onrender.com'
# assign the /status endpoint to another variable called status_url (1 line)
status_url = root_url + '/status'
# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(status_url)
# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print(req.text)
else:
    print("Request failed with status code:", req.status_code)

"Alive"


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted. The nice thing for us is that JSON files follow a format we already know in Python...dictionaries!

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [2]:
# Set the countries_url variable (1 line)
countries_url = root_url + '/countries'

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(countries_url)

# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()


# display the request's status code and the countries variable (1 line)
print(req.status_code, countries)


403 {'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [3]:
# Set the cookie_url variable (1 line)
cookie_url = root_url + '/cookie/'
# Query the enpoint, set the cookies variable and display it (2 lines)
cookies = requests.get(cookie_url)
print(cookies.json())


{'message': 'The cookie has been created'}


Try to query the countries endpoint using the cookie, save the output and print it.

In [4]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(countries_url)
# display the countries variable (1 line)
print(countries.json())

{'message': 'The cookie is missing'}


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [5]:
# Set the leaders_url variable (1 line)
leaders_url = root_url + '/leaders'

# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url)

# display the leaders variable (1 line)
print(leaders.json())


{'message': 'The cookie is missing'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [6]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
country = 'us'
leaders_url = root_url + '/leaders?country=' + country
cookies = requests.get(cookie_url).cookies


# assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, cookies = cookies)

# display the leaders variable (1 line)
print(leaders.json())


[{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'place_of_bi

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [7]:
# 4 lines
#cookies = requests.get(cookie_url).cookies
#line 1
countries = requests.get(countries_url, cookies=cookies)
#line 2
leaders_per_country = {}
#line 3
for n in range(len(countries.json())):
    #country = countries.json()[n]
    #leaders_url = root_url + '/leaders?country=' + country
    #leaders = requests.get(leaders_url, cookies = cookies)
    #line 4
    leaders_per_country[countries.json()[n]] = [(leader['first_name'], leader['last_name'], leader['wikipedia_url']) for leader in leaders.json()]

    

print(leaders_per_country)

{'be': [('George', 'Washington', 'https://en.wikipedia.org/wiki/George_Washington'), ('Barack', 'Obama', 'https://en.wikipedia.org/wiki/Barack_Obama'), ('Abraham', 'Lincoln', 'https://en.wikipedia.org/wiki/Abraham_Lincoln'), ('George', 'Bush', 'https://en.wikipedia.org/wiki/George_W._Bush'), ('Bill', 'Clinton', 'https://en.wikipedia.org/wiki/Bill_Clinton'), ('Joe', 'Biden', 'https://en.wikipedia.org/wiki/Joe_Biden'), ('Franklin', 'Roosevelt', 'https://en.wikipedia.org/wiki/Franklin_D._Roosevelt'), ('Andrew', 'Johnson', 'https://en.wikipedia.org/wiki/Andrew_Johnson'), ('Gerald', 'Ford', 'https://en.wikipedia.org/wiki/Gerald_Ford'), ('Richard', 'Nixon', 'https://en.wikipedia.org/wiki/Richard_Nixon'), ('Lyndon', 'Johnson', 'https://en.wikipedia.org/wiki/Lyndon_B._Johnson'), ('John', 'Kennedy', 'https://en.wikipedia.org/wiki/John_F._Kennedy'), ('Dwight', 'Eisenhower', 'https://en.wikipedia.org/wiki/Dwight_D._Eisenhower'), ('Ronald', 'Reagan', 'https://en.wikipedia.org/wiki/Ronald_Reagan'),

In [8]:
# or 1 line
leaders_per_country = {country: [(leader['first_name'], leader['last_name'], leader['wikipedia_url']) for leader in requests.get(root_url + '/leaders?country=' + country, cookies=cookies).json()] for country in countries.json()}

print(leaders_per_country)

{'be': [('Guy', 'Verhofstadt', 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt'), ('Yves', 'Leterme', 'https://nl.wikipedia.org/wiki/Yves_Leterme'), ('Herman', 'None', 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy'), ('Léon', 'Delacroix', 'https://nl.wikipedia.org/wiki/L%C3%A9on_Delacroix'), ('Henry', 'Carton', 'https://nl.wikipedia.org/wiki/Henri_Carton_de_Wiart'), ('Georges', 'Theunis', 'https://nl.wikipedia.org/wiki/Georges_Theunis'), ('Aloïs', 'Van de Vyvere', 'https://nl.wikipedia.org/wiki/Aloys_Van_de_Vyvere'), ('Prosper', 'Poullet', 'https://nl.wikipedia.org/wiki/Prosper_Poullet'), ('Henri', 'Jaspar', 'https://nl.wikipedia.org/wiki/Henri_Jaspar'), ('Jules', 'None', 'https://nl.wikipedia.org/wiki/Jules_Renkin'), ('Paul', 'Van Zeeland', 'https://nl.wikipedia.org/wiki/Paul_van_Zeeland'), ('Achille', 'Van Acker', 'https://nl.wikipedia.org/wiki/Achiel_Van_Acker'), ('Camille', 'Huysmans', 'https://nl.wikipedia.org/wiki/Camille_Huysmans'), ('Gaston', 'Eyskens', 'https://nl.wikipedia.

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [9]:
# < 15 lines

def get_leaders():
    root_url = 'https://country-leaders.onrender.com'
    cookie_url = root_url + '/cookie/'
    countries_url = root_url + '/countries'
        
    cookies = requests.get(cookie_url).cookies
    countries = requests.get(countries_url, cookies=cookies)
         
    leaders_per_country = {}
    
    for n in range(len(countries.json())):
        country = countries.json()[n]
        leaders_url = root_url + '/leaders?country=' + country
        leaders = requests.get(leaders_url, cookies=cookies)
        leaders_per_country[country] = [(leader['first_name'], leader['last_name'], leader['wikipedia_url']) for leader in leaders.json()]
    
    return leaders_per_country

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [10]:
# 2 lines
result = get_leaders()
print(result)

{'be': [('Guy', 'Verhofstadt', 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt'), ('Yves', 'Leterme', 'https://nl.wikipedia.org/wiki/Yves_Leterme'), ('Herman', 'None', 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy'), ('Léon', 'Delacroix', 'https://nl.wikipedia.org/wiki/L%C3%A9on_Delacroix'), ('Henry', 'Carton', 'https://nl.wikipedia.org/wiki/Henri_Carton_de_Wiart'), ('Georges', 'Theunis', 'https://nl.wikipedia.org/wiki/Georges_Theunis'), ('Aloïs', 'Van de Vyvere', 'https://nl.wikipedia.org/wiki/Aloys_Van_de_Vyvere'), ('Prosper', 'Poullet', 'https://nl.wikipedia.org/wiki/Prosper_Poullet'), ('Henri', 'Jaspar', 'https://nl.wikipedia.org/wiki/Henri_Jaspar'), ('Jules', 'None', 'https://nl.wikipedia.org/wiki/Jules_Renkin'), ('Paul', 'Van Zeeland', 'https://nl.wikipedia.org/wiki/Paul_van_Zeeland'), ('Achille', 'Van Acker', 'https://nl.wikipedia.org/wiki/Achiel_Van_Acker'), ('Camille', 'Huysmans', 'https://nl.wikipedia.org/wiki/Camille_Huysmans'), ('Gaston', 'Eyskens', 'https://nl.wikipedia.

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [11]:
# 3 lines
test_leader_wiki = result['be'][-1][2]
r = requests.get(test_leader_wiki).text
print(r)


<!DOCTYPE html>
<html class="client-nojs" lang="nl" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Sophie Wilmès - Wikipedia</title>
<script>document.documentElement.className="client-js";(function(){var cookie=document.cookie.match(/(?:^|; )nlwikimwclientprefs=([^;]+)/);if(cookie){var featureName=cookie[1];document.documentElement.className=document.documentElement.className.replace(featureName+'-enabled',featureName+'-disabled');}}());RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t.",".\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","januari","februari","maart","april","mei","juni","juli","augustus","september","oktober","november","december"],"wgRequestId":"cc49408c-a37b-49ba-b1c1-2cc7ef34fe79","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Sophie_Wilmès","wgTitle":"Sophie Wilmès","wgCurRevisionId":64388025,"wgRevisionId":64388025,"wgArticleId":4067917,"wgIsArti

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [12]:
# 3 lines

from bs4 import BeautifulSoup
soup = BeautifulSoup(r, 'lxml')

print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="nl">
 <head>
  <meta charset="utf-8"/>
  <title>
   Sophie Wilmès - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";(function(){var cookie=document.cookie.match(/(?:^|; )nlwikimwclientprefs=([^;]+)/);if(cookie){var featureName=cookie[1];document.documentElement.className=document.documentElement.className.replace(featureName+'-enabled',featureName+'-disabled');}}());RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t.",".\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","januari","februari","maart","april","mei","juni","juli","augustus","september","oktober","november","december"],"wgRequestId":"cc49408c-a37b-49ba-b1c1-2cc7ef34fe79","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Sophie_Wilmès","wgTitle":"Sophie Wilmès","wgCurRevisionId":64388025,"wgRevisionId":64388025,"wgArticleId

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [13]:
# 2 lines

paragraphs = soup.find_all('p')
    

print(paragraphs)



[<p><b>Sophie Wilmès</b> (<a href="/wiki/Elsene" title="Elsene">Elsene</a>, <a href="/wiki/15_januari" title="15 januari">15 januari</a> <a href="/wiki/1975" title="1975">1975</a>) is een <a class="mw-redirect" href="/wiki/Belgi%C3%AB_(hoofdbetekenis)" title="België (hoofdbetekenis)">Belgisch</a> <a href="/wiki/Politicus" title="Politicus">politica</a> van de <a href="/wiki/Mouvement_R%C3%A9formateur" title="Mouvement Réformateur">MR</a>. Van 27 oktober 2019 tot 1 oktober 2020 was ze de eerste vrouwelijke <a href="/wiki/Premier_van_Belgi%C3%AB" title="Premier van België">premier</a> van België.<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup> Sinds 1 oktober 2020 is ze vicepremier en minister van Buitenlandse Zaken en Buitenlandse Handel. Om familiale redenen legde ze op 21 april 2022 haar bevoegdheden als minister tijdelijk neer. Op 15 juli 2022 legde ze haar functie definitief neer.
</p>, <p>Wilmès is een dochter van <a href="/wiki/Philippe_Wilm%C3%A8s" titl

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [14]:
# <10 lines
first_paragraph = None
for x in paragraphs:
    if x.text.strip():
        first_paragraph = x.text
        break
    

print(first_paragraph)
    



Sophie Wilmès (Elsene, 15 januari 1975) is een Belgisch politica van de MR. Van 27 oktober 2019 tot 1 oktober 2020 was ze de eerste vrouwelijke premier van België.[1] Sinds 1 oktober 2020 is ze vicepremier en minister van Buitenlandse Zaken en Buitenlandse Handel. Om familiale redenen legde ze op 21 april 2022 haar bevoegdheden als minister tijdelijk neer. Op 15 juli 2022 legde ze haar functie definitief neer.



At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [15]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
    print(wikipedia_url)
#   [insert your code]
    r = requests.get(wikipedia_url).text
    soup = BeautifulSoup(r, 'lxml')
    paragraphs = soup.find_all('p')
    first_paragraph = None
    for x in paragraphs:        
        if x.text.strip():
            first_paragraph = x.text
            break
#   return first_paragraph
    return first_paragraph  

In [16]:
# Test: 3 lines

result_2 = get_first_paragraph(result['us'][0][2])
print(result_2)

https://en.wikipedia.org/wiki/George_Washington


George Washington (February 22, 1732[c] – December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Continental Congress as commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War and served as president of the Constitutional Convention of 1787, which created and ratified the Constitution of the United States and the American federal government. Washington has been called the "Father of his Country" for his manifold leadership in the nation's founding.[10]



### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [17]:
# 3 lines
import re
res = re.sub('\[\w?\d?|\]', '', result_2)
print(res)



George Washington (February 22, 1732 – December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Continental Congress as commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War and served as president of the Constitutional Convention of 1787, which created and ratified the Constitution of the United States and the American federal government. Washington has been called the "Father of his Country" for his manifold leadership in the nation's founding.



Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [18]:

# 10 lines
# def get_first_paragraph(wikipedia_url):
def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
    print(wikipedia_url)
#   [insert your code]
    r = requests.get(wikipedia_url).text
    soup = BeautifulSoup(r, 'lxml')
    paragraphs = soup.find_all('p')
    first_paragraph = None
    for x in paragraphs:        
        if x.text.strip():
            first_paragraph = x.text
            first_paragraph = re.sub('\[\w?\d?|\]', '', first_paragraph)
            break
#   return first_paragraph
    return first_paragraph  


result_2 = get_first_paragraph(result['us'][0][2])
print(result_2)

https://en.wikipedia.org/wiki/George_Washington
George Washington (February 22, 1732 – December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Continental Congress as commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War and served as president of the Constitutional Convention of 1787, which created and ratified the Constitution of the United States and the American federal government. Washington has been called the "Father of his Country" for his manifold leadership in the nation's founding.



Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [19]:
# < 20 lines
def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
    print(wikipedia_url)
#   [insert your code]
    r = requests.get(wikipedia_url).text
    soup = BeautifulSoup(r, 'lxml')
    paragraphs = soup.find_all('p')
    first_paragraph = None
    regex_list = ["\ *\(/.+/\[e\].*\)", "\ *\(/.+/.*\)", "\[[0-9]+\]", "[\n]", "[\t]", "[\xa0]", "\[\w?\d?|\]"]
    for x in paragraphs: 
        if x.find_parent(class_="bandeau-cell") or x.find_parent(class_="plainlist"):
            continue
        else:
            if x.text.strip():
                first_paragraph = x.text
                for n in range(len(regex_list)):
                    first_paragraph = re.sub(regex_list[n], '', first_paragraph)
                break
#   return first_paragraph
    return first_paragraph  
result_2 = get_first_paragraph(result['ru'][1][2])
print(result_2)

https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D0%B4%D0%B2%D0%B5%D0%B4%D0%B5%D0%B2,_%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%B8%D0%B9_%D0%90%D0%BD%D0%B0%D1%82%D0%BE%D0%BB%D1%8C%D0%B5%D0%B2%D0%B8%D1%87
Дми́трий Анато́льевич Медве́дев (род. 14 сентября 1965…, Ленинград, РСФСР, СССР)— российский государственный и политический деятель. Первый заместитель председателя Военно-промышленной комиссии Российской Федерации с 26 декабря 2022года. Заместитель председателя Совета Безопасности Российской Федерации с 16 января 2020года. Председатель партии «Единая Россия» с 26 мая 2012года.


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [20]:
# < 20 lines

def get_first_paragraph(wikipedia_url):
    
    r = requests.get(wikipedia_url).text
    soup = BeautifulSoup(r, 'lxml')
    paragraphs = soup.find_all('p')
    first_paragraph = None
    regex_list = ["\ *\(/.+/\[e\].*\)", "\ *\(/.+/.*\)", "\[[0-9]+\]", "[\n]", "[\t]", "[\xa0]", "\[\w?\d?|\]"]
    for x in paragraphs:
        if x.find_parent(class_="bandeau-cell") or x.find_parent(class_="plainlist"):
            continue
        else:
            if x.text.strip():
                first_paragraph = x.text
                for n in range(len(regex_list)):
                    first_paragraph = re.sub(regex_list[n], '', first_paragraph)
                break
    return first_paragraph

def get_leaders():
    root_url = 'https://country-leaders.onrender.com'
    cookie_url = root_url + '/cookie/'
    countries_url = root_url + '/countries'

    cookies = requests.get(cookie_url).cookies
    countries = requests.get(countries_url, cookies=cookies)

    leaders_per_country = {}

    for country in countries.json():
        leaders_url = root_url + '/leaders?country=' + country
        leaders = requests.get(leaders_url, cookies=cookies)
        leader_data = [(leader['first_name'], leader['last_name'], leader['wikipedia_url']) for leader in leaders.json()]
        leaders_per_country[country] = []

        for leader in leader_data:
            first_paragraph = get_first_paragraph(leader[2])
            leaders_per_country[country].append({
                'first_name': leader[0],
                'last_name': leader[1],
                'wikipedia_url': leader[2],
                'first_paragraph': first_paragraph
            })

    return leaders_per_country




In [21]:
# Check the output of your function (2 lines)
result = get_leaders()
print(result)

{'be': [{'first_name': 'Guy', 'last_name': 'Verhofstadt', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'first_paragraph': 'Guy Maurice Marie-Louise Verhofstadt (uitspraak(info / uitleg)) (Dendermonde, 11 april 1953) is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten (Open Vld). Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Nu is hij lid van het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa (ALDE) was.'}, {'first_name': 'Yves', 'last_name': 'Leterme', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'first_paragraph': 'Yves Camille Désiré Leterme (Wervik, 6 oktober 1960) is een Belgisch politicus. Hij was onder meer Vlaams minister-president en premier van België. Hij was lang boegbeeld van de CD&V en boegbeeld van het toenmalige kartel CD&V/N-VA.'}, {'first_name': 'Herman', 'last_name': 'None', 'wikipedia_url': 'https://nl.wi

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [22]:
# < 25 lines

def get_first_paragraph(wikipedia_url):
    
    r = requests.get(wikipedia_url).text
    soup = BeautifulSoup(r, 'lxml')
    paragraphs = soup.find_all('p')
    first_paragraph = None
    regex_list = ["\ *\(/.+/\[e\].*\)", "\ *\(/.+/.*\)", "\[[0-9]+\]", "[\n]", "[\t]", "[\xa0]", "\[\w?\d?|\]"]
    for x in paragraphs:
        if x.find_parent(class_="bandeau-cell") or x.find_parent(class_="plainlist"):
            continue
        else:
            if x.text.strip():
                first_paragraph = x.text
                for n in range(len(regex_list)):
                    first_paragraph = re.sub(regex_list[n], '', first_paragraph)
                break
    return first_paragraph

def get_leaders():
    root_url = 'https://country-leaders.onrender.com'
    cookie_url = root_url + '/cookie/'
    countries_url = root_url + '/countries'

    cookies = requests.get(cookie_url).cookies
    countries = requests.get(countries_url, cookies=cookies)

    leaders_per_country = {}

    for country in countries.json():
        leaders_url = root_url + '/leaders?country=' + country
        leaders = requests.get(leaders_url, cookies=cookies)
        leader_data = [(leader['first_name'], leader['last_name'], leader['wikipedia_url']) for leader in leaders.json()]
        leaders_per_country[country] = []

        for leader in leader_data:
            first_paragraph = get_first_paragraph(leader[2])
            leaders_per_country[country].append({
                'first_name': leader[0],
                'last_name': leader[1],
                'wikipedia_url': leader[2],
                'first_paragraph': first_paragraph
            })

    return leaders_per_country

Check the output of your function again.

In [23]:
# Check the output of your function (1 line)
result = get_leaders()
print(result)

{'be': [{'first_name': 'Guy', 'last_name': 'Verhofstadt', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'first_paragraph': 'Guy Maurice Marie-Louise Verhofstadt (uitspraak(info / uitleg)) (Dendermonde, 11 april 1953) is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten (Open Vld). Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Nu is hij lid van het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa (ALDE) was.'}, {'first_name': 'Yves', 'last_name': 'Leterme', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'first_paragraph': 'Yves Camille Désiré Leterme (Wervik, 6 oktober 1960) is een Belgisch politicus. Hij was onder meer Vlaams minister-president en premier van België. Hij was lang boegbeeld van de CD&V en boegbeeld van het toenmalige kartel CD&V/N-VA.'}, {'first_name': 'Herman', 'last_name': 'None', 'wikipedia_url': 'https://nl.wi

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [24]:
# < 20 lines



def get_first_paragraph(wikipedia_url, session):
    r = session.get(wikipedia_url).text
    soup = BeautifulSoup(r, 'lxml')
    paragraphs = soup.find_all('p')
    first_paragraph = None
    regex_list = ["\ *\(/.+/\[e\].*\)", "\ *\(/.+/.*\)", "\[[0-9]+\]", "[\n]", "[\t]", "[\xa0]", "\[\w?\d?|\]"]
    for x in paragraphs:
        if x.find_parent(class_="bandeau-cell") or x.find_parent(class_="plainlist"):
            continue
        else:
            if x.text.strip():
                first_paragraph = x.text
                for n in range(len(regex_list)):
                    first_paragraph = re.sub(regex_list[n], '', first_paragraph)
                break
    return first_paragraph




Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [25]:
# <25 lines
def get_leaders():
    root_url = 'https://country-leaders.onrender.com'
    cookie_url = root_url + '/cookie/'
    countries_url = root_url + '/countries'

    session = requests.Session()  # Create a session object
    cookies = session.get(cookie_url).cookies
    countries = session.get(countries_url, cookies=cookies)

    leaders_per_country = {}
    for country in countries.json():
        leaders_url = root_url + '/leaders?country=' + country
        leaders_response = session.get(leaders_url, cookies=cookies)
        if leaders_response.status_code == 200:
            leader_data = leaders_response.json()
            leaders_per_country[country] = []

            for leader in leader_data:
                first_paragraph = get_first_paragraph(leader['wikipedia_url'], session)  # Pass the session object to the function
                leaders_per_country[country].append({
                    'first_name': leader['first_name'],
                    'last_name': leader['last_name'],
                    'wikipedia_url': leader['wikipedia_url'],
                    'first_paragraph': first_paragraph
                })
        else:
            print(f"Error retrieving leaders for {country}")

    return leaders_per_country

Test your new functions.



In [26]:
result = get_leaders()
print(result)

{'be': [{'first_name': 'Guy', 'last_name': 'Verhofstadt', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'first_paragraph': 'Guy Maurice Marie-Louise Verhofstadt (uitspraak(info / uitleg)) (Dendermonde, 11 april 1953) is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten (Open Vld). Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Nu is hij lid van het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa (ALDE) was.'}, {'first_name': 'Yves', 'last_name': 'Leterme', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'first_paragraph': 'Yves Camille Désiré Leterme (Wervik, 6 oktober 1960) is een Belgisch politicus. Hij was onder meer Vlaams minister-president en premier van België. Hij was lang boegbeeld van de CD&V en boegbeeld van het toenmalige kartel CD&V/N-VA.'}, {'first_name': 'Herman', 'last_name': 'None', 'wikipedia_url': 'https://nl.wi

## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [27]:
# 3 lines
import json
with open('leaders.json', 'w') as fp:
    json.dump(leaders_per_country, fp)


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [31]:
# 3 lines
with open('leaders.json', 'r') as fp:
    data = json.load(fp)
print(data)

{'be': [{'first_name': 'Guy', 'last_name': 'Verhofstadt', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'first_paragraph': 'Guy Maurice Marie-Louise Verhofstadt (uitspraak(info / uitleg)) (Dendermonde, 11 april 1953) is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten (Open Vld). Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Nu is hij lid van het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa (ALDE) was.'}, {'first_name': 'Yves', 'last_name': 'Leterme', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'first_paragraph': 'Yves Camille Désiré Leterme (Wervik, 6 oktober 1960) is een Belgisch politicus. Hij was onder meer Vlaams minister-president en premier van België. Hij was lang boegbeeld van de CD&V en boegbeeld van het toenmalige kartel CD&V/N-VA.'}, {'first_name': 'Herman', 'last_name': 'None', 'wikipedia_url': 'https://nl.wi

Make a function `save(leaders_per_country)` to call this code easily.

In [29]:
# 3 lines
def save_json(leaders_per_country):
    with open('leaders.json', 'w') as fp:
        json.dump(leaders_per_country, fp)

In [30]:
# Call the function (1 line)
save_json(result)

## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

### Constraints

#### Code style

- Your code should be **commented**. You can use a docstring to explain what your functions are doing. E.g. 

```python
def add(number_one, number_two):
    """
    Function that will perform the add operation between two numbers (in params).
    """
    result = number_one + number_two
    return result
```


- Your code should be **cleaned of any commented, unused code**.

### GitHub Repository

- Push your code on a GitHub repository. The name is at the top of this page.
- Your repository should only contain the files specified.
- Your `README.md` should be nice and complete.
  It should include the following outline:
  - Description (What is the project about? What is the goal? What is your approach?)
  - Installation (What are the requirements to use your code?)
  - Usage (How does someone use your code?)
  - Timeline (How long did the project take?)
  - Personal situation (e.g. This project was done as part of the AI Boocamp at BeCode.org.)

Feel free to add more information.

## Evaluation criteria

| Criteria       | Indicator                                                   | Yes/No |
| -------------- | ----------------------------------------------------------- | ------ |
| 1. Is complete | The student has realized all must-have features.            |        |
|                | There is a published GitHub repo available.                 |        |
|                | The scrapper starts by running `python main.py` in the terminal |        |
| 2. Is correct  | The code is well typed.                                     |        |
|                | All the constraints are respected                           |        |


## To go further (if time permits)

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!

![wikepedia_gif](https://media.giphy.com/media/mADzY6J0QjniQl9gs5/giphy.gif)